In [ ]:
    PROJECT_ID = "cloud-llm-preview1"  # @param {type:"string"}
    LOCATION = "us-central1"  # @param {type:"string"}

    # Initialize Vertex AI
    import vertexai

    vertexai.init(project=PROJECT_ID, location=LOCATION)

In [371]:
import time
from vertexai.preview.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Image,
    Part,
)

In [372]:
multimodal_model = GenerativeModel("gemini-ultra-vision")
text_model= GenerativeModel("gemini-ultra")

In [373]:
contents="You are a data steward. Your role is to produce human meaningful metadata descriptions.\
Table's fully qualified name:bigquery:jsk-dataplex-demo-380508.metadata_generation.cc \
These SQL queries can be used to generate the data in table \
1. insert into metadata_generation.cc select * from metadata_generation.crimes;\
2. insert into metadata_generation.cc SELECT unique_key uk, case_number cn, date as dt, block b, iucr as i, primary_type as pt, description as de, location_description as ld, arrest as ar, domestic as d, beat as bt, district di, ward as wr, community_area as cs, fbi_code as fc, x_coordinate xc, y_coordinate yc, year yr, updated_on uo, latitude la, longitude lo, location ltn FROM `bigquery-public-data.chicago_crime.crime` ; \
3. create or replace table metadata_generation.cc as SELECT unique_key uk, case_number cn, date as dt, block b, iucr as i, primary_type as pt, description as de, location_description as ld, arrest as ar, domestic as d, beat as bt, district di, ward as wr, community_area as cs, fbi_code as fc, x_coordinate xc, y_coordinate yc, year yr, updated_on uo, latitude la, longitude lo, location ltn FROM `bigquery-public-data.chicago_crime.crime` ;\
4. create or replace table metadata_generation.cc as SELECT unique_key uk, case_number cn, date as dt, block b, iucr as i, primary_type as pt, description as de, location_description as ld, arrest as ar, domestic as d, beat as bt, district di, ward as wr, community_area as cs, fbi_code as fc, x_coordinate xc, y_coordinate yc, year yr, updated_on uo, latitude la, longitude lo, location ltn FROM `bigquery-public-data.chicago_crime.crime` LIMIT 1000 ;\
This is table profile information "+profil+". Provide brief description of table contents and usage"

In [374]:
responses = text_model.generate_content(contents,stream=True)

print("\n-------Response--------")



-------Response--------


In [375]:

for response in responses:
    print(response.text, end="")

The table **cc** contains data related to crimes, likely from the Chicago area. The data includes information such as the crime type, location, date, arrest status, and other relevant details. This table can be used for various purposes, including crime analysis, crime mapping, and identifying crime patterns.

### Let's list all tables we want to analyze


In [ ]:
table_list=[
    {'dataset': 'iowa_liquor_sales', 'table': 'sales'},
    {'dataset': 'london_bicycles', 'table': 'cycle_hire'},
    {'dataset': 'london_bicycles', 'table': 'cycle_stations'},
    {'dataset': 'ml_datasets', 'table': 'census_adult_income'},
    {'dataset': 'ml_datasets', 'table': 'credit_card_default'},
    {'dataset': 'ml_datasets', 'table': 'holidays_and_events_for_forecasting'},
    {'dataset': 'ml_datasets', 'table': 'iris'},
    {'dataset': 'ml_datasets', 'table': 'penguins'},
    {'dataset': 'ml_datasets', 'table': 'ulb_fraud_detection'},
    {'dataset': 'new_york_citibike', 'table': 'citibike_stations'},
    {'dataset': 'new_york_citibike', 'table': 'citibike_trips'},
    {'dataset': 'fdic_banks', 'table': 'institutions'},
    {'dataset': 'fdic_banks', 'table': 'locations'},
    {'dataset': 'fda_food', 'table': 'food_enforcement'},
    {'dataset': 'fda_food', 'table': 'food_events'},
    {'dataset': 'fcc_political_ads', 'table': 'broadcast_tv_radio_station'},
    {'dataset': 'fcc_political_ads', 'table': 'content_info'},
    {'dataset': 'fcc_political_ads', 'table': 'file_history'},
    {'dataset': 'fcc_political_ads', 'table': 'file_record'},
]

### Let's get schema for the table

In [ ]:
from google.cloud import bigquery
import requests
from google.oauth2 import service_account
import vertexai
from vertexai.language_models import TextGenerationModel
from google.cloud import datacatalog_lineage_v1 as lineage_v1


def _get_table_schema_bq(project_id, dataset_name, table_name):
    bqclient = bigquery.Client(project=project_id)
    """Get the schema of a table in BigQuery."""
    table = bqclient.get_table(f"{dataset_name}.{table_name}")
    return table.schema

print (_get_table_schema_bq('bigquery-public-data','ml_datasets','iris')) 

project_id='cloud-llm-preview1'

In [ ]:
def _get_upstream_tables(project_id,dataset,tablename,debug=0):
    # Create a client
    client = lineage_v1.LineageClient()
    target = lineage_v1.EntityReference()
    #source.fully_qualified_name = "bigquery:bigquery-public-data.ml_datasets.iris"
    target = lineage_v1.EntityReference()
    target.fully_qualified_name = "bigquery:"+project_id+"."+dataset+"."+tablename
    if debug==1:
        print("fully_qualified_name:"+target.fully_qualified_name)
    # Initialize request argument(s)
    request = lineage_v1.SearchLinksRequest(
        parent="projects/446454295341/locations/us",
        target=target,
    )

    # Make the request
    page_result = client.search_links(request=request)

    bigquery_tables=[]
    # Handle the response
    for response in page_result:
        if str(response.source.fully_qualified_name).startswith("bigquery:"):
            bigquery_tables.append(str(response.source.fully_qualified_name).replace("bigquery:",""))
        if debug==1:
            for x in bigquery_tables:
                print("substrings with names:"+x)
    return bigquery_tables


#sample_list_links()
tablesy=_get_upstream_tables('jsk-dataplex-demo-380508','taxi_data','taxi_trips',1)

print(tablesy)

In [314]:
#_get_table_schema('ml_datasets','iris')
import google.cloud.dataplex_v1 as dplx
import json
from google.protobuf.json_format import MessageToJson


dplx_client=dplx.DataScanServiceClient()


def _get_single_data_scanname(tableref):
    
    response=dplx_client.list_data_scans(parent="projects/jsk-dataplex-demo-380508/locations/us-central1")
    #print(response)
    for element in response:
        print("START HERE---->\n"+str(element))
        
        print(element.data.resource)
        print("element resource name:"+element.data.resource)
        print("string position:"+str(str(element.data.resource).find(tableref)))
        if str(element.data.resource).find(tableref) > -1:
            
            print("FOUND at ->"+str(str(element.data.resource).find(tableref))+"\n element name:"+element.name)
            print("END HERE---->\n")
            return element.name
        else: 
            print("NOT FOUND in :"+str(element.data.resource))
        print("END HERE---->\n")
        
#test='//bigquery.googleapis.com/projects/jsk-dataplex-demo-380508/datasets/metadata_generation/tables/cc'
#print(test.find('cc'))     
output=_get_single_data_scanname('cc')


START HERE---->
name: "projects/jsk-dataplex-demo-380508/locations/us-central1/dataScans/simple-scan"
uid: "19f630a2-3a94-427a-be62-77ab215ae133"
display_name: "simple scan"
state: ACTIVE
create_time {
  seconds: 1707298498
  nanos: 605295541
}
update_time {
  seconds: 1707298504
  nanos: 21725447
}
data {
  resource: "//bigquery.googleapis.com/projects/jsk-dataplex-demo-380508/datasets/taxi_data/tables/taxi_trips"
}
execution_spec {
  trigger {
    on_demand {
    }
  }
}
execution_status {
  latest_job_start_time {
    seconds: 1707298542
    nanos: 220000000
  }
  latest_job_end_time {
    seconds: 1707298573
    nanos: 872696270
  }
}
type_: DATA_PROFILE

//bigquery.googleapis.com/projects/jsk-dataplex-demo-380508/datasets/taxi_data/tables/taxi_trips
element resource name://bigquery.googleapis.com/projects/jsk-dataplex-demo-380508/datasets/taxi_data/tables/taxi_trips
string position:-1
NOT FOUND in ://bigquery.googleapis.com/projects/jsk-dataplex-demo-380508/datasets/taxi_data/tabl

In [346]:
#dplx_client.list_data_scans(parent="projects/jsk-dataplex-demo-380508/locations/us-central1")

def _get_table_profile(tablename):

        scanname=_get_single_data_scanname(tablename)


        requestscan=dplx.GetDataScanRequest(name=scanname)
        response_scan = dplx_client.get_data_scan(request=requestscan)
        # Set a GetDataScanRequest() using the args.datascan_name argument.

        dq_scan_name = response_scan.name
        jobs_request = dataplex_v1.ListDataScanJobsRequest(
                parent=scanname,
                page_size=10)
                # optional: filter, page_size

        page_result = dplx_client.list_data_scan_jobs(request=jobs_request)

        counter = 0
        job_names = []
        for response in page_result:
                counter += 1
                job_names.append(response.name)

        print('Jobs scanned: ' + str(counter))


        for job_name in job_names:
                job_request = dataplex_v1.GetDataScanJobRequest(
                        name=job_name,
                        view="FULL",
                )

                job_result = dplx_client.get_data_scan_job(request=job_request)
                        # Skips jobs if not in succeeded state
                if job_result.state != 4:
                        continue

                dq_job_id = job_result.uid

                print("dq_scan_name --> " + dq_scan_name)
                print("dq_job_id --> " + dq_job_id)
                print("job_result.data_quality_result.row_count --> " + str(job_result.data_quality_result.row_count))
                print("job_result.start_time --> " + str(job_result.start_time))
                print("job_result.end_time --> " + str(job_result.end_time))
                print("MessageToJson(job_result.data_profile_result.scanned_data._pb) --> " + MessageToJson(job_result.data_profile_result.scanned_data._pb))

                profile=MessageToJson(job_result.data_profile_result.profile._pb)
                return profile

                
profil=_get_table_profile('ccagg')
        

START HERE---->
name: "projects/jsk-dataplex-demo-380508/locations/us-central1/dataScans/simple-scan"
uid: "19f630a2-3a94-427a-be62-77ab215ae133"
display_name: "simple scan"
state: ACTIVE
create_time {
  seconds: 1707298498
  nanos: 605295541
}
update_time {
  seconds: 1707298504
  nanos: 21725447
}
data {
  resource: "//bigquery.googleapis.com/projects/jsk-dataplex-demo-380508/datasets/taxi_data/tables/taxi_trips"
}
execution_spec {
  trigger {
    on_demand {
    }
  }
}
execution_status {
  latest_job_start_time {
    seconds: 1707298542
    nanos: 220000000
  }
  latest_job_end_time {
    seconds: 1707298573
    nanos: 872696270
  }
}
type_: DATA_PROFILE

//bigquery.googleapis.com/projects/jsk-dataplex-demo-380508/datasets/taxi_data/tables/taxi_trips
element resource name://bigquery.googleapis.com/projects/jsk-dataplex-demo-380508/datasets/taxi_data/tables/taxi_trips
string position:-1
NOT FOUND in ://bigquery.googleapis.com/projects/jsk-dataplex-demo-380508/datasets/taxi_data/tabl

In [344]:
print(profil)

{
  "fields": [
    {
      "name": "cuk",
      "type": "INTEGER",
      "mode": "NULLABLE",
      "profile": {
        "distinctRatio": 5.245718182533507e-05,
        "topNValues": [
          {
            "value": "1",
            "count": "447073",
            "ratio": 0.977174568758251
          },
          {
            "value": "2",
            "count": "9557",
            "ratio": 0.0208888869460303
          },
          {
            "value": "3",
            "count": "571",
            "ratio": 0.001248043784261097
          },
          {
            "value": "4",
            "count": "129",
            "ratio": 0.000281957352311176
          },
          {
            "value": "5",
            "count": "61",
            "ratio": 0.00013332867047272665
          },
          {
            "value": "6",
            "count": "29",
            "ratio": 6.338576137227988e-05
          },
          {
            "value": "7",
            "count": "22",
            "ratio": 4.8

In [168]:
project_id = 'jsk-dataplex-demo-380508'

def build_context():
    profil=_get_table_profile('crimes')
    table_description=_get_table_bq_metadata('jsk-dataplex-demo-380508.metadata_generation.crimes')
    load_query=get_process_details('projects/446454295341/locations/us/processes/e8042f025185164c319acb1272629893')

    print("--------------Printing retrieved metadata: --------------")
    #print(profil)
    print(table_description.)
    #print(load_query)
build_context()

found at ->-1
 element name:projects/jsk-dataplex-demo-380508/locations/us-central1/dataScans/a77b3de83-e198-40e1-921a-d322c6e46eca
Jobs scanned: 1
dq_scan_name --> projects/jsk-dataplex-demo-380508/locations/us-central1/dataScans/a77b3de83-e198-40e1-921a-d322c6e46eca
dq_job_id --> 0129f297-acd6-49be-b9cb-47c2707a301f
job_result.data_quality_result.row_count --> 0
job_result.start_time --> 2024-02-05 11:10:45.121000+00:00
job_result.end_time --> 2024-02-05 11:11:12.330388+00:00
MessageToJson(job_result.data_profile_result.scanned_data._pb) --> {}
jsk-dataplex-demo-380508.metadata_generation.crimes None {'dataplex-dp-published-scan': 'a77b3de83-e198-40e1-921a-d322c6e46eca', 'dataplex-dp-published-project': 'jsk-dataplex-demo-380508', 'dataplex-dp-published-location': 'us-central1'}
US:bquxjob_fbd9c05_18d78ec937e
Type: query
State: DONE
Created: 2024-02-05T10:59:47.151000+00:00
Query: create or replace table metadata_generation.cc as
SELECT unique_key uk, 
case_number cn, 
date as dt, 
b

In [164]:
def _get_table_bq_metadata(table_ref):
    bqclient = bigquery.Client(project=project_id)
    table = bqclient.get_table(table_ref)

    print(table,table.description,table.labels)
    return(table)

_get_table_bq_metadata('bigquery-public-data.ml_datasets.iris')
print(_get_table_bq_metadata('jsk-dataplex-demo-380508.taxi_data.source_table'))

NameError: name 'project_id' is not defined

In [ ]:
profile_json=json.loads(profile)
for i in profile_json['fields']:
    print(i['name'])
    print(i['type'])
    print(i['mode'])
    print(i['profile'])


In [186]:
from  google.cloud import datacatalog_v1


def _search_data_catalog(phrase):
    dc_client=datacatalog_v1.DataCatalogClient()
    project_id='jsk-dataplex-demo-380508'
    search_string = phrase
    scope = datacatalog_v1.types.SearchCatalogRequest.Scope()
    scope.include_project_ids.append(project_id)
    search_results = dc_client.search_catalog(scope=scope, query=search_string)

    results_list = []
    for result in search_results:
        result_dict = json.loads(MessageToJson(result._pb))
        result_dict['image_url'] = "https://lh3.googleusercontent.com/p9ST3mhfKqDdxwwgyGHCFmCddgFeHnYlQfCbORDHJm48z1cZhEknPXlbY_iGsnr2sIPk8EVanoqGjA=s48-w48-rw-lo" # Add the image_url to the dictionary
        results_list.append(result_dict)
        #results_list.append(result)
    return results_list

output=_search_data_catalog('taxi')


In [189]:
for i in output:
    #print(i._pb)
    print(i)
    break

{'searchResultType': 'ENTRY', 'searchResultSubtype': 'entry.table', 'relativeResourceName': 'projects/jsk-dataplex-demo-380508/locations/us/entryGroups/@bigquery/entries/cHJvamVjdHMvanNrLWRhdGFwbGV4LWRlbW8tMzgwNTA4L2RhdGFzZXRzL3RheGlfZGF0YS90YWJsZXMvdGF4aV90cmlwcw', 'linkedResource': '//bigquery.googleapis.com/projects/jsk-dataplex-demo-380508/datasets/taxi_data/tables/taxi_trips', 'modifyTime': '2024-02-02T08:50:21Z', 'integratedSystem': 'BIGQUERY', 'fullyQualifiedName': 'bigquery:jsk-dataplex-demo-380508.taxi_data.taxi_trips', 'image_url': 'https://lh3.googleusercontent.com/p9ST3mhfKqDdxwwgyGHCFmCddgFeHnYlQfCbORDHJm48z1cZhEknPXlbY_iGsnr2sIPk8EVanoqGjA=s48-w48-rw-lo'}


In [200]:
def _get_entry_details(entry_id):
     dc_client=datacatalog_v1.DataCatalogClient()
     request=datacatalog_v1.LookupEntryRequest(linked_resource = entry_id)
     results=dc_client.lookup_entry(request=request)
 #   entry = dc_client.lookup_entry(request={"linked_resource": entry_id})
 #   entry_dict = json.loads(MessageToJson(entry._pb))
     return results

output=_get_entry_details('//bigquery.googleapis.com/projects/jsk-dataplex-demo-380508/datasets/taxi_data/tables/taxi_trips')

In [201]:
field_json=MessageToJson(output._pb)
print(MessageToJson(output._pb))

{
  "name": "projects/jsk-dataplex-demo-380508/locations/us/entryGroups/@bigquery/entries/cHJvamVjdHMvanNrLWRhdGFwbGV4LWRlbW8tMzgwNTA4L2RhdGFzZXRzL3RheGlfZGF0YS90YWJsZXMvdGF4aV90cmlwcw",
  "type": "TABLE",
  "schema": {
    "columns": [
      {
        "type": "STRING",
        "description": "Test",
        "mode": "NULLABLE",
        "column": "TaxiCompany"
      },
      {
        "type": "INT64",
        "description": "Dupa",
        "mode": "NULLABLE",
        "column": "Vendor_Id"
      },
      {
        "type": "TIMESTAMP",
        "description": "why not",
        "mode": "NULLABLE",
        "column": "Pickup_DateTime"
      },
      {
        "type": "TIMESTAMP",
        "description": "Tu tez cos jest",
        "mode": "NULLABLE",
        "column": "Dropoff_DateTime"
      },
      {
        "type": "STRING",
        "mode": "NULLABLE",
        "column": "Store_And_Forward"
      },
      {
        "type": "INT64",
        "mode": "NULLABLE",
        "column": "Rate_Code_Id

In [ ]:
def _get_table _dataplex_profile(table_ref):
    bqclient = bigquery.Client(project=project_id)
    table = bqclient.get_table(table_ref)

    print(table,table.description,table.labels)
    return(table)1

In [ ]:
from google.cloud import datacatalog_lineage_v1

def sample_list_processes():
    # Create a client
    client = datacatalog_lineage_v1.LineageClient()

    # Initialize request argument(s)
    request = datacatalog_lineage_v1.ListProcessesRequest(
        parent="projects/446454295341/locations/us",

    )

    # Make the request
    page_result = client.list_processes(request=request)

    # Handle the response
    for response in page_result:
        print(response)
        print(MessageToJson(response._pb))
        
sample_list_processes()

In [ ]:
from google.cloud import bigquery


def _bq_job_details(job_name):
    client = bigquery.Client
    location = "us"
    job_id = "bquxjob_35b0569c_18d59fa2d41"

    bqclient=bigquery.Client(project='jsk-dataplex-demo-380508')
 

    job = client.get_job(bqclient,job_id=job_id, location=location)

    # All job classes have "location" and "job_id" string properties.
    # Use these properties for job operations such as "cancel_job" and
    # "delete_job".
    print(f"{job.location}:{job.job_id}")
    print(f"Type: {job.job_type}")
    print(f"State: {job.state}")
    print(f"Created: {job.created.isoformat()}")
    print(f"Query: {job.query}")

_bq_job_details('bquxjob_35b0569c_18d59fa2d41')

In [ ]:
from google.cloud import datacatalog_lineage_v1

def sample_list_runs():
    # Create a client
    client = datacatalog_lineage_v1.LineageClient()

    # Initialize request argument(s)
    request = datacatalog_lineage_v1.ListRunsRequest(
        parent="projects/446454295341/locations/us/processes/64d98e962706f0223f3e957fedf5af2b-73c9499597cc075e4716c4ecfeb5c058",

    )

    # Make the request
    page_result = client.list_runs(request=request)

    # Handle the response
    for response in page_result:
        print(response)
        print(MessageToJson(response._pb)
        
sample_list_runs()

In [ ]:
def sample_list_events():
    # Create a client
    client = datacatalog_lineage_v1.LineageClient()

    # Initialize request argument(s)
    request = datacatalog_lineage_v1.ListLineageEventsRequest(
        parent="projects/446454295341/locations/us/processes/64d98e962706f0223f3e957fedf5af2b-73c9499597cc075e4716c4ecfeb5c058/runs/2024-01-04T11:34:02.440095Z",

    )

    # Make the request
    page_result = client.list_lineage_events(request=request)

    # Handle the response
    for response in page_result:
        print(response)
        print(MessageToJson(response._pb))
        
sample_list_events()


In [275]:
from google.cloud import dataplex_v1 #import lineage_v1
from google.cloud import datacatalog_lineage_v1 as lineage_v1


def lineage_source_tables(target_fqdn):
    # Create a client
    client = lineage_v1.LineageClient()
    target = lineage_v1.EntityReference()
    #source.fully_qualified_name = "bigquery:bigquery-public-data.ml_datasets.iris"
    target = lineage_v1.EntityReference()
    target.fully_qualified_name = target_fqdn

    # Initialize request argument(s)
    request = lineage_v1.SearchLinksRequest(
        parent="projects/446454295341/locations/us",
        target=target,
    )

    print('lineage_source_tables: looking for lineage links for table:'+target_fqdn)
    # Make the request
    page_result = client.search_links(request=request)
    response_list=[]
  
    # Handle the response
    for response in page_result:
        new_element={}
        new_element['name']=response.name
        new_element['source']=response.source.fully_qualified_name
        new_element['target']=response.target.fully_qualified_name
        print('lineage_source_tables: found lineage link:'+response.name)
        response_list.append(new_element)

   # print(json.loads(MessageToJson(page_result._pb)))
    return MessageToJson(page_result._pb)

def find_lineage_processes(target_table):
    #create a client
    client = lineage_v1.LineageClient()
    #client.batch_search_link_processes()
    #initialize request arguments
    print('find_lineage_processes: looking for lineage links for table:'+target_table)
    response=json.loads(lineage_source_tables(target_table))
    print('find_lineage_processes: response->'+json.dumps(response))
    links=[]
    for i in response['links']:
        links.append(i['name'])
        print('find_lineage looking for link:' + i['name'])
    request = lineage_v1.BatchSearchLinkProcessesRequest(parent="projects/446454295341/locations/us",links=links)
    #make the request

    page_result = client.batch_search_link_processes(request=request)
    return MessageToJson(page_result._pb)
    
#find_lineage_processes("bigquery:jsk-dataplex-demo-380508.taxi_data.taxi_trips")
#output=lineage_source_tables("bigquery:jsk-dataplex-demo-380508.metadata_generation.cc")
second_output=find_lineage_processes("bigquery:jsk-dataplex-demo-380508.metadata_generation.cc")
print(second_output)

find_lineage_processes: looking for lineage links for table:bigquery:jsk-dataplex-demo-380508.metadata_generation.cc
lineage_source_tables: looking for lineage links for table:bigquery:jsk-dataplex-demo-380508.metadata_generation.cc
lineage_source_tables: found lineage link:projects/446454295341/locations/us/links/p:f761823fd78e615f5209fe3279279c4b
lineage_source_tables: found lineage link:projects/446454295341/locations/us/links/p:59317e61604dae41b426b68d79a5eff5
find_lineage_processes: response->{"links": [{"name": "projects/446454295341/locations/us/links/p:f761823fd78e615f5209fe3279279c4b", "source": {"fullyQualifiedName": "bigquery:jsk-dataplex-demo-380508.metadata_generation.crimes"}, "target": {"fullyQualifiedName": "bigquery:jsk-dataplex-demo-380508.metadata_generation.cc"}, "startTime": "2024-02-06T21:24:11.877Z", "endTime": "2024-02-06T21:24:16.209Z"}, {"name": "projects/446454295341/locations/us/links/p:59317e61604dae41b426b68d79a5eff5", "source": {"fullyQualifiedName": "big

In [278]:
def get_dependencies_info(tablename):
    processes=[]
    
    response=json.loads(find_lineage_processes(tablename))
    for i in response['processLinks']:

        print('found process through lineage: '+i['process'])
        processes.append(i['process'])

    job_details=[]    
    for i in processes:
        result=get_process_details(i)
        job_details.append(result)
    return job_details

get_dependencies_info('bigquery:jsk-dataplex-demo-380508.metadata_generation.cc')

find_lineage_processes: looking for lineage links for table:bigquery:jsk-dataplex-demo-380508.metadata_generation.cc
lineage_source_tables: looking for lineage links for table:bigquery:jsk-dataplex-demo-380508.metadata_generation.cc
lineage_source_tables: found lineage link:projects/446454295341/locations/us/links/p:f761823fd78e615f5209fe3279279c4b
lineage_source_tables: found lineage link:projects/446454295341/locations/us/links/p:59317e61604dae41b426b68d79a5eff5
find_lineage_processes: response->{"links": [{"name": "projects/446454295341/locations/us/links/p:f761823fd78e615f5209fe3279279c4b", "source": {"fullyQualifiedName": "bigquery:jsk-dataplex-demo-380508.metadata_generation.crimes"}, "target": {"fullyQualifiedName": "bigquery:jsk-dataplex-demo-380508.metadata_generation.cc"}, "startTime": "2024-02-06T21:24:11.877Z", "endTime": "2024-02-06T21:24:16.209Z"}, {"name": "projects/446454295341/locations/us/links/p:59317e61604dae41b426b68d79a5eff5", "source": {"fullyQualifiedName": "big

[name: "projects/446454295341/locations/us/processes/6a02b6b0bb60e248ec976390f1eb0960"
 display_name: "Query"
 attributes {
   key: "bigquery_job_id"
   value {
     string_value: "bquxjob_33997b3c_18d804e98fb"
   }
 }
 origin {
   source_type: BIGQUERY
   name: "jsk-dataplex-demo-380508:us.bquxjob_33997b3c_18d804e98fb"
 },
 name: "projects/446454295341/locations/us/processes/460ff7a90c353ea0a4ace3e5f0301cdb"
 display_name: "Query"
 attributes {
   key: "bigquery_job_id"
   value {
     string_value: "bquxjob_d7f14f1_18d80451cf8"
   }
 }
 origin {
   source_type: BIGQUERY
   name: "jsk-dataplex-demo-380508:us.bquxjob_d7f14f1_18d80451cf8"
 },
 name: "projects/446454295341/locations/us/processes/ea1975c3dcf35d4d1483166d80a1e70b"
 display_name: "Query"
 attributes {
   key: "bigquery_parent_job_id"
   value {
     string_value: "bquxjob_65b4eabe_18d78f45c17"
   }
 }
 attributes {
   key: "bigquery_job_id"
   value {
     string_value: "script_job_748f0c7e7d40117a5b901afb44d86710_0"
   }
 

In [280]:
from google.cloud import bigquery


def _bq_job_details(job_name):
    client = bigquery.Client
    location = "us"
    job_id = job_name#"bquxjob_35b0569c_18d59fa2d41"

    bqclient=bigquery.Client(project='jsk-dataplex-demo-380508')
 

    job = client.get_job(bqclient,job_id=job_id, location=location)

    # All job classes have "location" and "job_id" string properties.
    # Use these properties for job operations such as "cancel_job" and
    # "delete_job".
    print(f"{job.location}:{job.job_id}")
    print(f"Type: {job.job_type}")
    print(f"State: {job.state}")
    print(f"Created: {job.created.isoformat()}")
    print(f"Query: {job.query}")
    return job


def get_process_details (process_name):
    # Create a client
    client = datacatalog_lineage_v1.LineageClient()

    # Initialize request argument(s)
    request = datacatalog_lineage_v1.GetProcessRequest(
        name=process_name,
    )

    # Make the request
    response = client.get_process(request=request)
   # print(response)
    jsonized=json.loads(MessageToJson(response._pb))
    bq_sql=_bq_job_details(jsonized['attributes']['bigquery_job_id'])
    print("bq_sql.query:"+bq_sql.query)

    
   # if response.attributes[key=="bigquery_job_id":
   #     print(response.attributes.value.string_value)
    # Handle the response
   # print(response)
  #  print(MessageToJson(response._pb))
    return bq_sql

get_process_details('projects/446454295341/locations/us/processes/807c39109ee0a29b59bf5b6bc96dfdb8')

US:bquxjob_35b0569c_18d59fa2d41
Type: query
State: DONE
Created: 2024-01-30T10:46:24.851000+00:00
Query: insert into jsk-dataplex-demo-380508.taxi_data.taxi_trips select * from jsk-dataplex-demo-380508.taxi_data.source_table
bq_sql.query:insert into jsk-dataplex-demo-380508.taxi_data.taxi_trips select * from jsk-dataplex-demo-380508.taxi_data.source_table


QueryJob<project=jsk-dataplex-demo-380508, location=US, id=bquxjob_35b0569c_18d59fa2d41>

In [345]:
print(profil)

{
  "fields": [
    {
      "name": "cuk",
      "type": "INTEGER",
      "mode": "NULLABLE",
      "profile": {
        "distinctRatio": 5.245718182533507e-05,
        "topNValues": [
          {
            "value": "1",
            "count": "447073",
            "ratio": 0.977174568758251
          },
          {
            "value": "2",
            "count": "9557",
            "ratio": 0.0208888869460303
          },
          {
            "value": "3",
            "count": "571",
            "ratio": 0.001248043784261097
          },
          {
            "value": "4",
            "count": "129",
            "ratio": 0.000281957352311176
          },
          {
            "value": "5",
            "count": "61",
            "ratio": 0.00013332867047272665
          },
          {
            "value": "6",
            "count": "29",
            "ratio": 6.338576137227988e-05
          },
          {
            "value": "7",
            "count": "22",
            "ratio": 4.8

In [341]:
def _get_prompt(tablename,profile,sql_queries):
    prompt="You are a data steward. Your role is to produce human meaningful metadata descriptions.\
    Table's fully qualified "+tablename+" \
    These SQL queries can be used to generate the data in table \
    "+ sql_queries+"\
    This is table profile information "+profile+"\
    Provide  description of table contents and usage. \
    Provide description of each column with top common values and their explanation. \
    Provide pseudocode expressed in SQL to calculate each field formatted in LaTeX.\
    "
    return prompt

def generate_metadata()

In [342]:
responses = text_model.generate_content(contents,stream=True)

print("\n-------Response--------")

for response in responses:
    print(response.text, end="")


-------Response--------
**Table Description and Usage:**

The `ccagg` table contains aggregated crime data from the city of Chicago. It provides insights into the distribution of crime incidents across months, days, hours, crime types, and geographical areas. This data can be used for crime analysis, resource allocation, and policy decisions.

**Pseudocode for Field Calculations:**

```sql
-- Count of Unique Crime Keys (cuk)
SELECT count(distinct unique_key) AS cuk

-- Month (m)
SELECT extract(month from date) AS m

-- Day (d)
SELECT extract(day from date) AS d

-- Hour (h)
SELECT extract(hour from date) AS h

-- Primary Crime Type (pt)
SELECT primary_type AS pt

-- Arrest (ar)
SELECT CASE WHEN arrest = 'true' THEN TRUE ELSE FALSE END AS ar

-- Domestic Violence (dc)
SELECT CASE WHEN domestic = 'true' THEN TRUE ELSE FALSE END AS dc

-- Beat (bt)
SELECT beat AS bt

-- District (di)
SELECT district AS di

-- Community Area (cs)
SELECT community_area AS cs

-- FBI Code (fc)
SELECT fbi_co

In [366]:
from flask import Flask, request, render_template
from  metadata.metadata import _search_data_catalog,_get_entry_details,get_dependencies_info,_get_table_profile,generate_table_description


In [8]:
from flask import Flask, request, render_template
from  metadata.metadata import _search_data_catalog,_get_entry_details,get_dependencies_info,_get_table_profile,generate_table_description


entry_id = "//bigquery.googleapis.com/projects/jsk-dataplex-demo-380508/datasets/metadata_generation/tables/ccagg"


result = _get_entry_details(entry_id)  # Replace this with your function to get a result by ID
print("generate():result['fullyQualifiedName']:",result['fullyQualifiedName'])
additional_info=get_dependencies_info(result['fullyQualifiedName'])
profile_info=_get_table_profile(entry_id)


Python(87552) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


generate():result['fullyQualifiedName']: bigquery:jsk-dataplex-demo-380508.metadata_generation.ccagg


Python(87621) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


find_lineage_processes: looking for lineage links for table:bigquery:jsk-dataplex-demo-380508.metadata_generation.ccagg


Python(87661) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


lineage_source_tables: looking for lineage links for table:bigquery:jsk-dataplex-demo-380508.metadata_generation.ccagg
find_lineage_processes: response->{}
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/jskuratowicz/Projects/Metadata Generation/.venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/94/dc5cg25d1kb01n91j57br5zh00gtw4/T/ipykernel_56984/3286515443.py", line 10, in <module>
    additional_info=get_dependencies_info(result['fullyQualifiedName'])
  File "/Users/jskuratowicz/Projects/Metadata Generation/metadata/metadata.py", line 250, in get_dependencies_info
    print("MessageToJson(job_result.data_profile_result.scanned_data._pb) --> " + MessageToJson(job_result.data_profile_result.scanned_data._pb))
  File "/Users/jskuratowicz/Projects/Metadata Generation/metadata/metadata.py", line 163, in find_lineage_processes
    client = lineage_v1.LineageClient()
KeyError: 'links'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/jskuratowicz/Projects/Met

In [383]:
def _get_prompt(tablename,profile,sql_queries):
    queries=""
    for i in sql_queries:
        queries=queries+i.query+";"

    contents="You are a data steward. Your role is to produce human meaningful metadata descriptions.\
    Table's fully qualified "+tablename+" \
    These SQL queries can be used to generate the data in table \
    "+ queries +"\
    This is table profile information "+profile+"\
    Provide  description of table contents and usage. \
    Provide description of each column with top common values and their explanation. \
    Provide pseudocode expressed in SQL and format it in LaTeX.\
    "
    contents2 = """
Prompt:

You are a highly skilled data steward. Your primary role is to produce comprehensive, human-understandable metadata descriptions for accurate data analysis and interpretation.

Inputs:

* Table's fully qualified name:  """+tablename+""" 
* SQL queries used to generate data: """+queries+"""
* Table profile information: """+profile+"""

Tasks:

1. Thorough Table Description:  Provide a detailed description of the table's contents and its typical usage scenarios. Explain the relationship and purpose of the data within the table in a clear and concise manner.

2. Insightful Column Analysis: Describe each column individually. This should include:
    * Column Name: 
    * Data Type:
    * Top Common Values:  List the most frequently occurring values.
    * Value Explanations: Provide clear interpretations of what these common values represent in the real-world context of the data.
    * SQL Pseudocode with LaTeX Output: Express the core logic represented in the column data using SQL-like pseudocode. Ensure that the pseudocode is formatted correctly using LaTeX syntax for visually clear mathematical and logical expressions.

Output Format:

Please structure the output in a well-organized JSON format for easy parsing and integration into other systems.

Example JSON Structure:

```json
{
    \"table_description\": \"...\”, 
    \"column_analysis\": [
        {
            \"column_name\": \"...\",
            \"data_type\": \"...\",
            \"top_values\": [\"...\", \"...\"],
            \"value_explanations\": [\"...\", \"...\"]
            \"column_pseudocode\": \"...\",
        },
        /* ...other columns */
    ]
}
"""


    return contents2

def generate_table_description(tablename,profile,sql_queries):

    prompt=_get_prompt(tablename,profile,sql_queries)

    text_model= GenerativeModel("gemini-ultra")
    
    responses = text_model.generate_content(prompt,stream=True)
    print("\n-------Response--------")
    for response in responses:
        print(response.text, end="")
    return responses

generate_table_description(tablename=result['fullyQualifiedName'],profile=profile_info,\
                                                     sql_queries=additional_info)


-------Response--------
```json
{
  "table_description": "The table `ccagg` contains aggregated crime statistics for the city of Chicago. It is generated from the `bigquery-public-data.chicago_crime.crime` dataset by grouping the data based on the following dimensions: month, day, hour, primary crime type, arrest status, domestic violence status, beat, district, community area, FBI code, and year. The table provides insights into crime patterns and trends over time and across different geographic areas.",
  "column_analysis": [
    {
      "column_name": "cuk",
      "data_type": "INTEGER",
      "top_values": [
        "1",
        "2",
        "3"
      ],
      "value_explanations": [
        "Represents the number of unique crimes.",
        "Indicates that there were two unique crimes.",
        "Indicates that there were three unique crimes."
      ],
      "column_pseudocode": "`SUM(DISTINCT unique_key)`"
    },
    {
      "column_name": "m",
      "data_type": "INTEGER",
    

<generator object _GenerativeModel._generate_content_streaming at 0x12e795580>

In [5]:
from  metadata.metadata import _search_data_catalog,_get_entry_details,get_dependencies_info,_get_table_profile,generate_table_description


#"bigquery:jsk-dataplex-demo-380508.metadata_generation.ccagg"
entry_id="//bigquery.googleapis.com/projects/jsk-dataplex-demo-380508/datasets/metadata_generation/tables/ccagg"
result = _get_entry_details(entry_id)  # Replace this with your function to get a result by ID
additional_info=get_dependencies_info(result['fullyQualifiedName'])
profile_info=_get_table_profile(entry_id)

generated_description=generate_table_description(tablename=result['fullyQualifiedName'],profile=profile_info,\
                                                     sql_queries=additional_info)

/Users/jskuratowicz/Projects/Metadata Generation/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


find_lineage_processes: looking for lineage links for table:bigquery:jsk-dataplex-demo-380508.metadata_generation.ccagg
lineage_source_tables: looking for lineage links for table:bigquery:jsk-dataplex-demo-380508.metadata_generation.ccagg
lineage_source_tables: found lineage link:projects/446454295341/locations/us/links/p:0116a1f484c3479853a431f90f6a696d
find_lineage_processes: response->{"links": [{"name": "projects/446454295341/locations/us/links/p:0116a1f484c3479853a431f90f6a696d", "source": {"fullyQualifiedName": "bigquery:bigquery-public-data.chicago_crime.crime"}, "target": {"fullyQualifiedName": "bigquery:jsk-dataplex-demo-380508.metadata_generation.ccagg"}, "startTime": "2024-02-05T11:08:23.637Z", "endTime": "2024-02-05T11:08:35.671Z"}]}
find_lineage looking for link:projects/446454295341/locations/us/links/p:0116a1f484c3479853a431f90f6a696d
get_dependencies_info:found process through lineage: projects/446454295341/locations/us/processes/5c66d1d1e5c06e9a6e405e63cbad2378
US:scri

TypeError: 'GenerationResponse' object is not iterable

In [23]:
from  metadata.metadata import _get_entry_details,_search_data_catalog,_get_entry_details,get_dependencies_info,_get_table_profile,generate_table_description
import json

entry_id = "//bigquery.googleapis.com/projects/jsk-dataplex-demo-380508/datasets/metadata_generation/tables/cc"



result = _get_entry_details(entry_id)  # Replace this with your function to get a result by ID
#print("generate():result['fullyQualifiedName']:",result['fullyQualifiedName'])
#additional_info=get_dependencies_info(result['fullyQualifiedName'])

print(json.dumps(result, indent=4))



Python(42363) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


{
    "name": "projects/jsk-dataplex-demo-380508/locations/us/entryGroups/@bigquery/entries/cHJvamVjdHMvanNrLWRhdGFwbGV4LWRlbW8tMzgwNTA4L2RhdGFzZXRzL21ldGFkYXRhX2dlbmVyYXRpb24vdGFibGVzL2Nj",
    "type": "TABLE",
    "description": "This is bq description",
    "schema": {
        "columns": [
            {
                "type": "INT64",
                "mode": "NULLABLE",
                "column": "uk"
            },
            {
                "type": "STRING",
                "mode": "NULLABLE",
                "column": "cn"
            },
            {
                "type": "TIMESTAMP",
                "mode": "NULLABLE",
                "column": "dt"
            },
            {
                "type": "STRING",
                "mode": "NULLABLE",
                "column": "b"
            },
            {
                "type": "STRING",
                "mode": "NULLABLE",
                "column": "i"
            },
            {
                "type": "STRING",
        

In [26]:
try:
    dataplexOverview=result['businessContext']['entryOverview']['overview']
except:
    dataplexOverview=""
print("dataplexOverview:",dataplexOverview)


try:
    bqOveview=result['description']
except:
    bqOveview=""
print("bqOveview:",bqOveview)

dataplexOverview: <p>This is an overview</p>
<ol>
<li>a</li>
<li>b</li>
<li>c</li>
</ol>
bqOveview: This is bq description


In [5]:
from google.cloud import bigquery
import requests
from google.oauth2 import service_account
import vertexai
from vertexai.language_models import TextGenerationModel
from google.cloud import datacatalog_lineage_v1 as lineage_v1
from google.cloud import datacatalog_v1
import json
from google.protobuf.json_format import MessageToJson


dc_client=datacatalog_v1.DataCatalogClient()
request=datacatalog_v1.LookupEntryRequest(linked_resource = "//bigquery.googleapis.com/projects/jsk-dataplex-demo-380508/datasets/metadata_generation/tables/ccagg")
results=dc_client.lookup_entry(request=request)
#   entry = dc_client.lookup_entry(request={"linked_resource": entry_id})
#   entry_dict = json.loads(MessageToJson(entry._pb))
results_json = json.loads(MessageToJson(results._pb))

In [7]:
print(results_json)

{'name': 'projects/jsk-dataplex-demo-380508/locations/us/entryGroups/@bigquery/entries/cHJvamVjdHMvanNrLWRhdGFwbGV4LWRlbW8tMzgwNTA4L2RhdGFzZXRzL21ldGFkYXRhX2dlbmVyYXRpb24vdGFibGVzL2NjYWdn', 'type': 'TABLE', 'schema': {'columns': [{'type': 'INT64', 'mode': 'NULLABLE', 'column': 'cuk'}, {'type': 'INT64', 'mode': 'NULLABLE', 'column': 'm'}, {'type': 'INT64', 'mode': 'NULLABLE', 'column': 'd'}, {'type': 'INT64', 'mode': 'NULLABLE', 'column': 'h'}, {'type': 'STRING', 'mode': 'NULLABLE', 'column': 'pt'}, {'type': 'BOOL', 'mode': 'NULLABLE', 'column': 'ar'}, {'type': 'BOOL', 'mode': 'NULLABLE', 'column': 'dc'}, {'type': 'INT64', 'mode': 'NULLABLE', 'column': 'bt'}, {'type': 'INT64', 'mode': 'NULLABLE', 'column': 'di'}, {'type': 'INT64', 'mode': 'NULLABLE', 'column': 'cs'}, {'type': 'STRING', 'mode': 'NULLABLE', 'column': 'fc'}, {'type': 'INT64', 'mode': 'NULLABLE', 'column': 'yr'}]}, 'sourceSystemTimestamps': {'createTime': '2024-03-20T09:18:51.211Z', 'updateTime': '2024-03-20T09:18:51.212Z'}

In [1]:
from  metadata.metadata import _get_full_glossary

glossary=_get_full_glossary("dupa")
print(glossary)

/Users/jskuratowicz/Projects/Metadata Generation/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[{'term': 'Incident Type', 'description': "The primary classification of the crime as defined by the Chicago Police Department (e.g., 'THEFT', 'BATTERY', 'HOMICIDE')"}, {'term': 'Incident Category', 'description': "A broader grouping of the crime type (e.g., 'Property Crime', 'Violent Crime', 'Criminal Sexual Assault')"}, {'term': 'Arrest', 'description': "Whether an arrest was made in connection with the incident (Boolean: 'true' or 'false')"}, {'term': 'Domestic', 'description': "Whether the crime involved a domestic relationship between the victim and offender (Boolean: 'true' or 'false')"}, {'term': 'Beat', 'description': 'The police beat number where the crime occurred. Beats are smaller divisions within police districts.'}, {'term': 'District', 'description': 'The police district number where the crime occurred.'}, {'term': 'Community Area', 'description': 'The designated community area (out of 77) where the crime took place. Provides broader geographic context than beat or distr

In [ ]:
def _get_prompt_suggest_terms(columns,tablename,profile,sql_queries,glossary):
    queries=""
    for i in sql_queries:
        queries=queries+i.query+";"

    contents="You are a data steward and expert on data processing and SQL. Your role is to be a data steward and suggest the right glossary terms from available terms to match the table description\
    Table's fully qualified name:"+tablename+" \
    These SQL queries are used to generate the data in table \
    "+ queries +"\
    This is table profile information "+profile+"\
    These are columns and their description: "+columns+"\
    These are available glossary terms:"+glossary+"\
    do not use markdown, or do not the keyword 'sql' at the beginning, do not use ``` to enclose the code.\
    You must only use the terms from available glossary terms\
        "
    return contents

In [34]:
from google.cloud import bigquery

def _get_table_schema_bq(project_id, dataset_name, table_name):
    bqclient = bigquery.Client(project=project_id)
    """Get the schema of a table in BigQuery."""
    table = bqclient.get_table(f"{dataset_name}.{table_name}")
    return table.schema

print(_get_table_schema_bq('jsk-dataplex-demo-380508','metadata_generation','crimes'))

[SchemaField('unique_key', 'INTEGER', 'NULLABLE', None, None, (), PolicyTagList(names=('projects/jsk-dataplex-demo-380508/locations/us/taxonomies/8518413592929443054/policyTags/7658502391738002058',))), SchemaField('case_number', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('date', 'TIMESTAMP', 'NULLABLE', None, None, (), None), SchemaField('block', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('iucr', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('primary_type', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('description', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('location_description', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('arrest', 'BOOLEAN', 'NULLABLE', None, None, (), None), SchemaField('domestic', 'BOOLEAN', 'NULLABLE', None, None, (), None), SchemaField('beat', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('district', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('ward', 'INTEG

In [27]:
import re

def _read_entryid(entry_id="//bigquery.googleapis.com/projects/jsk-dataplex-demo-380508/datasets/metadata_generation/tables/crimes"):

    pattern = r".*/tables/(.+)"  # Capture everything after the last '/'
    match = re.search(pattern, entry_id)

    if match:
        tablename = match.group(1)
        #print(tablename)  # Output: ccagg
        
    else:
        print("Table name not found") 
    pattern_ds = r".*/datasets/(.+)/tables"  # Capture everything after the last '/'
    match = re.search(pattern_ds, entry_id)

    if match:
        dataset = match.group(1)
        #print(tablename)  # Output: ccagg
        
    else:
        print("Dataset name not found")    

    pattern_pr = r".*/projects/(.+)/datasets"  # Capture everything after the last '/'
    match = re.search(pattern_pr, entry_id)

    if match:
        project = match.group(1)
        #print(tablename)  # Output: ccagg
        
    else:
        print("Project name not found")   
    return project,dataset,tablename
print(_read_entryid())



def _read_tablename(tablename="bigquery:jsk-dataplex-demo-380508.metadata_generation.ccagg"):


    pattern = r"^bigquery:([^.]+)\.([^.]+)\.([^.]+)" 
    match = re.search(pattern, tablename)

    if match:
        project = match.group(1)
        dataset = match.group(2)
        table = match.group(3)             
        return project,dataset,table
    else:
        return None,None,None



print(_read_tablename())



('jsk-dataplex-demo-380508', 'metadata_generation', 'crimes')
('jsk-dataplex-demo-380508', 'metadata_generation', 'ccagg')


In [35]:
schema=_get_table_schema_bq('jsk-dataplex-demo-380508','metadata_generation','crimes')


In [50]:
import json
from google.protobuf.json_format import MessageToJson


for field in schema:
    print(field.name+" "+field.field_type)


unique_key INTEGER
case_number STRING
date TIMESTAMP
block STRING
iucr STRING
primary_type STRING
description STRING
location_description STRING
arrest BOOLEAN
domestic BOOLEAN
beat INTEGER
district INTEGER
ward INTEGER
community_area INTEGER
fbi_code STRING
x_coordinate FLOAT
y_coordinate FLOAT
year INTEGER
updated_on TIMESTAMP
latitude FLOAT
longitude FLOAT
location STRING


'INTEGER'

In [56]:
def _get_table_bq(project_id, dataset_name, table_name):
    bqclient = bigquery.Client(project=project_id)
    """Get the schema of a table in BigQuery."""
    table = bqclient.get_table(f"{dataset_name}.{table_name}")
    return table

project_id,dataset_name,table_name=_read_entryid()
table=_get_table_bq(project_id,dataset_name,table_name)

In [60]:
def _read_tablename(tablename="jsk-dataplex-demo-380508.metadata_generation.ccagg"):


    pattern = r"^([^.]+)\.([^.]+)\.([^.]+)" 
    match = re.search(pattern, tablename)

    if match:
        project = match.group(1)
        dataset = match.group(2)
        table = match.group(3)             
        return project,dataset,table
    else:
        return None,None,None 

def _read_columnname(tablename="jsk-dataplex-demo-380508.metadata_generation.ccagg.cuk"):


    pattern = r"^([^.]+)\.([^.]+)\.([^.]+)\.([^.]+)" 
    match = re.search(pattern, tablename)

    if match:
        project = match.group(1)
        dataset = match.group(2)
        table = match.group(3) 
        column = match.group(4)             
        return project,dataset,table,column
    else:
        return None,None,None,None
print(_read_tablename())
print(_read_columnname())

('jsk-dataplex-demo-380508', 'metadata_generation', 'ccagg')
('jsk-dataplex-demo-380508', 'metadata_generation', 'ccagg', 'cuk')


In [28]:
table.schema

NameError: name 'table' is not defined

In [57]:
from  google.cloud import datacatalog_v1beta1
dc_client=datacatalog_v1beta1.PolicyTagManagerClient()

request = datacatalog_v1beta1.GetPolicyTagRequest(name='projects/jsk-dataplex-demo-380508/locations/us/taxonomies/8518413592929443054/policyTags/7658502391738002058')

tax_request=datacatalog_v1beta1.GetTaxonomyRequest(name='projects/jsk-dataplex-demo-380508/locations/us/taxonomies/8518413592929443054')

response = dc_client.get_policy_tag(request=request)
response_tax=dc_client.get_taxonomy(request=tax_request)

In [14]:

print(response_tax.display_name+":"+response.display_name)

partner_training:can_see


In [84]:
source_column='''{"columns":["jsk-dataplex-demo-380508.metadata_generation.cc.uk"]}'''
try:
    cols=json.loads(source_column)
    cols_values = cols['columns']

except:
    cols=None

for i in cols_values:
    proj,dataset,table,column=_read_columnname(i)
    print(_read_columnname(i))
    table=_get_table_bq(proj,dataset,table)
    for col in table.schema:
        if(col.name==column):
            print(col.name+" "+col.field_type)
            print(col.policy_tags)

('jsk-dataplex-demo-380508', 'metadata_generation', 'cc', 'uk')
uk INTEGER
None


In [95]:
json.loads('''{"columns":["bigquery-public-data.chicago_crime.crime.iucr","jsk-dataplex-demo-380508.metadata_generation.crimes.i","jsk-dataplex-demo-380508.metadata_generation.crimes.iucr"]}''')



{'columns': ['bigquery-public-data.chicago_crime.crime.iucr',
  'jsk-dataplex-demo-380508.metadata_generation.crimes.i',
  'jsk-dataplex-demo-380508.metadata_generation.crimes.iucr']}

In [141]:


def get_display_name(tag):
    request = datacatalog_v1beta1.GetPolicyTagRequest(name=tag.names[0])

    response = dc_client.get_policy_tag(request=request)

    def get_taxonomy_from_string(string):
        pattern = r"^(.+)/policyTags"
        match = re.search(pattern, string)
        if match:
            taxonomy_id = match.group(1)
            return taxonomy_id
        else:
            return None

    taxonomy_name=get_taxonomy_from_string(response.name)

    tax_request=datacatalog_v1beta1.GetTaxonomyRequest(name=taxonomy_name)
    response_taxonomy=dc_client.get_taxonomy(request=tax_request)
    return response_taxonomy.display_name+":"+response.display_name

def get_policy_tags_for_columns(column_id='''{"columns":["bigquery-public-data.chicago_crime.crime.date","jsk-dataplex-demo-380508.metadata_generation.crimes.unique_key"]}'''):
    col_vals = json.loads(column_id)['columns']
    string=""

    for i in col_vals:
        proj,dataset,table,column=_read_columnname(i)
        print(_read_columnname(i))
        table=_get_table_bq(proj,dataset,table)
        for col in table.schema:
            if(col.name==column):
                if col.policy_tags != None:
                    string+=i+":"+get_display_name(col.policy_tags)+';'
    return string

tag=get_policy_tags_for_columns()

('bigquery-public-data', 'chicago_crime', 'crime', 'date')
('jsk-dataplex-demo-380508', 'metadata_generation', 'crimes', 'unique_key')


In [142]:
print(tag)

#print(response_tax.display_name+":"+response.display_name)

jsk-dataplex-demo-380508.metadata_generation.crimes.unique_key:partner_training:can_see;


In [153]:

from flask import Flask, request, jsonify
import base64
from vertexai.preview.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models    
from pdf2image import convert_from_bytes
from PIL import Image
import io

def combine_images(base64_images):
    images = [Image.open(io.BytesIO(base64.b64decode(img))) for img in base64_images]
    
    # Calculate the total height and the maximum width of the combined image
    total_height = sum(image.height for image in images)
    max_width = max(image.width for image in images)
    
    # Create a new image with the appropriate height and width
    combined_image = Image.new('RGB', (max_width, total_height))
    
    # Paste each image into the combined image
    y_offset = 0
    for image in images:
        combined_image.paste(image, (0, y_offset))
        y_offset += image.height
    
    # Convert the combined image back to base64
    img_byte_arr = io.BytesIO()
    combined_image.save(img_byte_arr, format='JPEG')  # Change to 'PNG' if preferred
    img_byte_arr = img_byte_arr.getvalue()
    base64_combined_image = base64.b64encode(img_byte_arr).decode('utf-8')
    
    return base64_combined_image

def pdf_to_images(base64_pdf):
    # Decode the base64 PDF
    pdf_bytes = base64.b64decode(base64_pdf)
    # Convert PDF to a list of images
    images = convert_from_bytes(pdf_bytes)
    # Convert images to base64
    base64_images = []
    for image in images:
        img_byte_arr = io.BytesIO()
        image.save(img_byte_arr, format='JPEG')  # You can choose JPEG or PNG depending on your needs
        img_byte_arr = img_byte_arr.getvalue()
        base64_encoded = base64.b64encode(img_byte_arr).decode('utf-8')
        base64_images.append(base64_encoded)
    return base64_images


with open("static/ChicagoChrimesTable.pdf", "rb") as file:
    pdf_content = file.read()



#pdf_content = file.read()
base64_images = pdf_to_images(base64.b64encode(pdf_content).decode('utf-8'))

# Decode the base64-encoded image to get raw image data
image_data = base64.b64decode(base64_images)


# Combine images into one
base64_combined_image = combine_images(image_data)  


# Create a part for the image using its raw data
image_part = Part.from_data(data=base64_combined_image, mime_type="image/jpeg")  # Ensure this matches the format you saved (JPEG or PNG)
text_part = Part.from_text(text_prompt)
returned_text=""
# Generate content based on the image
responses = model.generate_content(
    [image_part, text_prompt],
    generation_config={
        "max_output_tokens": 2048,
        "temperature": 0.4,
        "top_p": 1,
        "top_k": 32
    },
    safety_settings={
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    },
    stream=True,
)

# Collect responses
for i in responses:        
    returned_text+=i.text
return returned_text

TypeError: argument should be a bytes-like object or ASCII string, not 'list'

In [ ]:
import pytesseract
from PIL import Image

# Open the image file
image = Image.open('image.jpg')

# Convert the image to grayscale
image = image.convert('L')

# Use Tesseract to extract text from the image
text = pytesseract.image_to_string(image)

# Print the extracted text
print(text)
